In [74]:
#Value choosen abritrarily
WORD_NGRAMS_N = 3
BYTE_NGRAM_N = 4

In [2]:
from os import listdir
from os.path import isfile, join
from collections import Counter

import os
import json
import pandas as pd
import math
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import string
import re

In [76]:
txt_files = [f for f in listdir("raw-data") if isfile(join("raw-data", f)) and f.endswith(".txt")]
print(txt_files)

['741710208501547161.txt', '384021699466756103.txt']


In [77]:

def read_raw_data(filename):
    raw = ""
    with open(filename, 'r') as f:
        raw = f.read()
    #trim until find the first '{' or '['
    start = 0
    for i in range(len(raw)):
        if raw[i] == '{' or raw[i] == '[':
            start = i
            break
    raw = raw[start:]
    jason = json.loads(raw)
    return jason
print(len(read_raw_data(f"raw-data/{txt_files[0]}")))

157809


In [78]:


def to_dataframe(jason):
    df = pd.DataFrame(jason)
    #set header to predefined columns if header length == 5
    if len(df.columns) == 5:
        df.columns = ["author_id", "username", "channel_id", "created_at", "content"]
    return df

In [79]:

def filter_dataframe(df):
    #filter out rows with content length > 1000
    df = df[df["content"].str.len() <= 1000]
    #filter out content that has too much newline
    df = df[df["content"].str.count("\n") <= 10]
    #remove URL from content
    df["content"] = df["content"].apply(lambda x: re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", "", x))
    #filter out rows with empty content
    df = df[df["content"] != ""]
    #filter null values
    df = df[df["content"].notnull()]
    return df

In [80]:
test = to_dataframe(read_raw_data(f"raw-data/{txt_files[0]}"))
test = filter_dataframe(test)
print(len(test))
test.head()

KeyboardInterrupt: 

we good eh, we going to batch convert to CSV leh

In [ ]:
csv_files = [f for f in listdir("csv-data") if isfile(join("csv-data", f)) and f.endswith(".csv")]
print(f"Total csv files: {len(csv_files)}")

all_data = pd.DataFrame()
for file in csv_files:
    data = pd.read_csv(f"csv-data/{file}")
    all_data = pd.concat([all_data, data])
print(f"Total rows: {len(all_data)}")
all_data.head()


Total csv files: 2
Total rows: 2308890


,author_id,username,channel_id,created_at,content
0,548962858088792094,stoltzld,741710209902444554,1689447697,Thank you. I may have figured that out eventu...
1,709904412033810533,balam314,741710209902444554,1689446869,I suggest setting it to 10%
2,709904412033810533,balam314,741710209902444554,1689446855,"this is the item transfer range, you can remov..."
3,548962858088792094,stoltzld,741710209902444554,1689442578,During the last upgrade (July 13th) I ended up...
4,244966266845134858,buthed010203,741710209902444554,1689396296,I believe that's currently only sent when runn...


In [ ]:
#GC ?
data = None
test = None
csv_files = None
txt_files = None
file = None
del data, test, csv_files, txt_files, file

In [ ]:
def GetSentences(text):
	"""
	Splits a given text into sentences based on the presence of '.', '?' or '!' characters.

	Args:
	text (str): The text to split into sentences.

	Returns:
	A list of sentences.
	"""
	result = [s.strip() for s in re.split('[\.\?\!]', text)]
	#remove empty strings
	result = [s for s in result if s]
	return result

def GetWords(text):
	"""
	Splits a given text into words based on non-word characters.

	Args:
	text (str): The text to split into words.

	Returns:
	A list of words.
	"""
	result = re.split('\W+', text)
	#remove empty strings
	result = [s for s in result if s]
	return result

def GetPunctuations(text):
	"""
	Extracts all punctuation characters from a given text.

	Args:
	text (str): The text to extract punctuation characters from.

	Returns:
	A string containing all punctuation characters.
	"""
	return "".join(re.findall(r'[' + string.punctuation + ']', text))

def GetByteNGrams(text, n):
	"""
	Extracts all byte n-grams from a given text.

	Args:
	text (str): The text to extract byte n-grams from.
	n (int): The length of each byte n-gram.

	Returns:
	A list of byte n-grams.
	"""
	return [text[i:i+n] for i in range(0, len(text)-n+1)]

def GetWordNGram(words, n):
	"""
	Extracts all word n-grams from a given list of words.

	Args:
	words (list): The list of words to extract word n-grams from.
	n (int): The length of each word n-gram.

	Returns:
	A list of word n-grams.
	"""
	return [" ".join(words[i:i+n]) for i in range(0, len(words)-n+1)]

# Test GetSentences
text = "This is a sentence. This is another sentence? This is a third sentence!"
expected = ['This is a sentence', 'This is another sentence', 'This is a third sentence']
result = GetSentences(text)
assert result == expected, f"Expected {expected}, got {result}"

# Test GetWords
text = "This is a sentence. This is another sentence? This is a third sentence!"
expected = ['This', 'is', 'a', 'sentence', 'This', 'is', 'another', 'sentence', 'This', 'is', 'a', 'third', 'sentence']
result = GetWords(text)
assert result == expected, f"Expected {expected}, got {result}"

# Test GetPunctuations
text = "This is a sentence. This is another sentence? This is a third sentence!"
expected = ".?!"
result = GetPunctuations(text)
assert result == expected, f"Expected {expected}, got {result}"

# Test GetByteNGrams
text = "This is a sentence."
expected = ['Th', 'hi', 'is', 's ', ' i', 'is', 's ', ' a', 'a ', ' s', 'se', 'en', 'nt', 'te', 'en', 'nc', 'ce', 'e.']
result = GetByteNGrams(text, 2)
assert result == expected, f"Expected {expected}, got {result}"

# Test GetWordNGram
words = ['This', 'is', 'a', 'sentence', 'This', 'is', 'another', 'sentence', 'This', 'is', 'a', 'third', 'sentence']
expected = ['This is', 'is a', 'a sentence', 'sentence This', 'This is', 'is another', 'another sentence', 'sentence This', 'This is', 'is a', 'a third', 'third sentence']
result = GetWordNGram(words, 2)
assert result == expected, f"Expected {expected}, got {result}"


In [ ]:
#Garbage Collector
words = None
text = None
result = None
expected = None
del words, text, result, expected

In [ ]:
#remove row that has no content
all_data = all_data[all_data["content"].notnull()]
#get row that has no content
no_content_row = all_data[all_data["content"].isnull()]
no_content_row

,author_id,username,channel_id,created_at,content


In [ ]:
#author_id	username	channel_id	created_at	content
#0	1	1	1	2019-01-01 00:00:00	Hello World

#sort authors by number of messages
authors = all_data.groupby("author_id").count().sort_values(by="content", ascending=False)
#get unique value count from each author
authors["username_count"] = all_data.groupby("author_id")["username"].nunique()
authors["channel_id_count"] = all_data.groupby("author_id")["channel_id"].nunique()
authors["created_at_count"] = all_data.groupby("author_id")["created_at"].nunique()

#get authors username single
authors["username"] = all_data.groupby("author_id")["username"].unique().apply(lambda x: x[0])

authors.head()

,username,channel_id,created_at,content,username_count,channel_id_count,created_at_count
author_id,,,,,,,
456226577798135808,Deleted User,190274,190274,190274,1,10,188466
651361636657135617,flattocatto,72738,72738,72738,1,1,72504
244966266845134858,buthed010203,53471,53471,53471,1,12,52980
723940197456281640,archange3965,40709,40709,40709,1,1,40461
559568621312147476,.deepfriedrice6828,38502,38502,38502,1,1,38419


In [ ]:
#column cleanup

authors.drop(columns=["channel_id", "created_at", "username_count", "channel_id_count", "created_at_count"], inplace=True)
authors.head()

,username,content
author_id,,
456226577798135808,Deleted User,190274
651361636657135617,flattocatto,72738
244966266845134858,buthed010203,53471
723940197456281640,archange3965,40709
559568621312147476,.deepfriedrice6828,38502


In [ ]:
#get shortest and longest message
authors["longest_message"] = all_data.groupby("author_id")["content"].apply(lambda x: x.str.len().max())
authors.head()

,username,content,longest_message
author_id,,,
456226577798135808,Deleted User,190274,892
651361636657135617,flattocatto,72738,975
244966266845134858,buthed010203,53471,966
723940197456281640,archange3965,40709,354
559568621312147476,.deepfriedrice6828,38502,717


In [ ]:

#get senteces, word, punctuations, byte n-grams, word n-grams from content for each author

#sentences
# sentences length for each row
all_data["sentences_len"] = all_data["content"].apply(lambda x: len(GetSentences(x)))
data_group = all_data.groupby("author_id")["sentences_len"]
#sum of sentences length
authors["sentences_sum"] = data_group.apply(lambda x: np.sum(x))
#mean of sentences length
authors["sentences_mean"] = data_group.apply(lambda x: np.mean(x))
#median of sentences length
authors["sentences_median"] = data_group.apply(lambda x: np.median(x))

authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median
author_id,,,,,,
456226577798135808,Deleted User,190274,892,196409,1.032243,1.0
651361636657135617,flattocatto,72738,975,73934,1.016443,1.0
244966266845134858,buthed010203,53471,966,57685,1.078809,1.0
723940197456281640,archange3965,40709,354,41425,1.017588,1.0
559568621312147476,.deepfriedrice6828,38502,717,39281,1.020233,1.0


In [ ]:
#words
# words length for each row
all_data["words_len"] = all_data["content"].apply(lambda x: len(GetWords(x)))
data_group = all_data.groupby("author_id")["words_len"]
#sum of words length
authors["words_sum"] = data_group.apply(lambda x: np.sum(x))
#mean of words length
authors["words_mean"] = data_group.apply(lambda x: np.mean(x))
#median of words length
authors["words_median"] = data_group.apply(lambda x: np.median(x))
authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median,words_sum,words_mean,words_median
author_id,,,,,,,,,
456226577798135808,Deleted User,190274,892,196409,1.032243,1.0,865550,4.548966,3.0
651361636657135617,flattocatto,72738,975,73934,1.016443,1.0,334413,4.597501,4.0
244966266845134858,buthed010203,53471,966,57685,1.078809,1.0,309528,5.788708,4.0
723940197456281640,archange3965,40709,354,41425,1.017588,1.0,145847,3.582672,2.0
559568621312147476,.deepfriedrice6828,38502,717,39281,1.020233,1.0,202842,5.268350,4.0


In [ ]:
#punctuations
# punctuations length for each row
all_data["punctuations_len"] = all_data["content"].apply(lambda x: len(GetPunctuations(x)))
data_group = all_data.groupby("author_id")["punctuations_len"]
#sum of punctuations length
authors["punctuations_sum"] = data_group.apply(lambda x: np.sum(x))
#mean of punctuations length
authors["punctuations_mean"] = data_group.apply(lambda x: np.mean(x))
#median of punctuations length
authors["punctuations_median"] = data_group.apply(lambda x: np.median(x))
authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median,words_sum,words_mean,words_median,punctuations_sum,punctuations_mean,punctuations_median
author_id,,,,,,,,,,,,
456226577798135808,Deleted User,190274,892,196409,1.032243,1.0,865550,4.548966,3.0,112389,0.590669,0.0
651361636657135617,flattocatto,72738,975,73934,1.016443,1.0,334413,4.597501,4.0,20764,0.285463,0.0
244966266845134858,buthed010203,53471,966,57685,1.078809,1.0,309528,5.788708,4.0,57263,1.070917,0.0
723940197456281640,archange3965,40709,354,41425,1.017588,1.0,145847,3.582672,2.0,15595,0.383085,0.0
559568621312147476,.deepfriedrice6828,38502,717,39281,1.020233,1.0,202842,5.268350,4.0,9079,0.235806,0.0


In [ ]:
#byte n-grams
# byte n-grams length for each row

all_data["byte_ngrams_len"] = all_data["content"].apply(lambda x: len(GetByteNGrams(x, BYTE_NGRAM_N)))
data_group = all_data.groupby("author_id")["byte_ngrams_len"]
#sum of byte n-grams length
authors["byte_ngrams_sum"] = data_group.apply(lambda x: np.sum(x))
#mean of byte n-grams length
authors["byte_ngrams_mean"] = data_group.apply(lambda x: np.mean(x))
#median of byte n-grams length
authors["byte_ngrams_median"] = data_group.apply(lambda x: np.median(x))
authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median,words_sum,words_mean,words_median,punctuations_sum,punctuations_mean,punctuations_median,byte_ngrams_sum,byte_ngrams_mean,byte_ngrams_median
author_id,,,,,,,,,,,,,,,
456226577798135808,Deleted User,190274,892,196409,1.032243,1.0,865550,4.548966,3.0,112389,0.590669,0.0,3841701,20.190362,14.0
651361636657135617,flattocatto,72738,975,73934,1.016443,1.0,334413,4.597501,4.0,20764,0.285463,0.0,1448522,19.914240,15.0
244966266845134858,buthed010203,53471,966,57685,1.078809,1.0,309528,5.788708,4.0,57263,1.070917,0.0,1510140,28.242225,22.0
723940197456281640,archange3965,40709,354,41425,1.017588,1.0,145847,3.582672,2.0,15595,0.383085,0.0,608244,14.941266,8.0
559568621312147476,.deepfriedrice6828,38502,717,39281,1.020233,1.0,202842,5.268350,4.0,9079,0.235806,0.0,883023,22.934471,17.0


In [ ]:
#word n-grams
# word n-grams length for each row
all_data["word_ngrams_len"] = all_data["content"].apply(lambda x: len(GetWordNGram(GetWords(x), WORD_NGRAMS_N)))
data_group = all_data.groupby("author_id")["word_ngrams_len"]
#sum of word n-grams length
authors["word_ngrams_sum"] = data_group.apply(lambda x: np.sum(x))
#mean of word n-grams length
authors["word_ngrams_mean"] = data_group.apply(lambda x: np.mean(x))
#median of word n-grams length
authors["word_ngrams_median"] = data_group.apply(lambda x: np.median(x))
authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median,words_sum,words_mean,words_median,punctuations_sum,punctuations_mean,punctuations_median,byte_ngrams_sum,byte_ngrams_mean,byte_ngrams_median,word_ngrams_sum,word_ngrams_mean,word_ngrams_median
author_id,,,,,,,,,,,,,,,,,,
456226577798135808,Deleted User,190274,892,196409,1.032243,1.0,865550,4.548966,3.0,112389,0.590669,0.0,3841701,20.190362,14.0,537939,2.827181,1.0
651361636657135617,flattocatto,72738,975,73934,1.016443,1.0,334413,4.597501,4.0,20764,0.285463,0.0,1448522,19.914240,15.0,208410,2.865215,2.0
244966266845134858,buthed010203,53471,966,57685,1.078809,1.0,309528,5.788708,4.0,57263,1.070917,0.0,1510140,28.242225,22.0,214009,4.002338,2.0
723940197456281640,archange3965,40709,354,41425,1.017588,1.0,145847,3.582672,2.0,15595,0.383085,0.0,608244,14.941266,8.0,80056,1.966543,0.0
559568621312147476,.deepfriedrice6828,38502,717,39281,1.020233,1.0,202842,5.268350,4.0,9079,0.235806,0.0,883023,22.934471,17.0,134606,3.496078,2.0


In [ ]:
authors[['username', 'sentences_mean', 'words_mean', 'punctuations_mean', 'byte_ngrams_mean', 'word_ngrams_mean']].head()

,username,sentences_mean,words_mean,punctuations_mean,byte_ngrams_mean,word_ngrams_mean
author_id,,,,,,
456226577798135808,Deleted User,1.032243,4.548966,0.590669,20.190362,2.827181
651361636657135617,flattocatto,1.016443,4.597501,0.285463,19.914240,2.865215
244966266845134858,buthed010203,1.078809,5.788708,1.070917,28.242225,4.002338
723940197456281640,archange3965,1.017588,3.582672,0.383085,14.941266,1.966543
559568621312147476,.deepfriedrice6828,1.020233,5.268350,0.235806,22.934471,3.496078


In [ ]:
authors['avg_sentence_length'] = authors['sentences_mean']
authors['avg_words_per_sentence'] = authors['words_mean'] / authors['sentences_mean']
authors['avg_punctuations_per_sentence'] = authors['punctuations_mean'] / authors['sentences_mean']

authors[['username', 'avg_sentence_length', 'avg_words_per_sentence', 'avg_punctuations_per_sentence']].head()

,username,avg_sentence_length,avg_words_per_sentence,avg_punctuations_per_sentence
author_id,,,,
456226577798135808,Deleted User,1.032243,4.406875,0.572219
651361636657135617,flattocatto,1.016443,4.523129,0.280845
244966266845134858,buthed010203,1.078809,5.365832,0.992684
723940197456281640,archange3965,1.017588,3.520748,0.376463
559568621312147476,.deepfriedrice6828,1.020233,5.163871,0.231130


In [ ]:


#count words, byte n-grams, word n-grams for author
for author_id, group in all_data.groupby("author_id"):
    #words
    words = GetWords(" ".join(group["content"]))
    words_count = Counter(words).most_common(50)
    authors.loc[author_id, "words_count"] = json.dumps(words_count)
    #byte n-grams
    byte_ngrams = GetByteNGrams(" ".join(group["content"]), BYTE_NGRAM_N)
    byte_ngrams_count = Counter(byte_ngrams).most_common(50)
    authors.loc[author_id, "byte_ngrams_count"] = json.dumps(byte_ngrams_count)
    #word n-grams
    word_ngrams = GetWordNGram(words, WORD_NGRAMS_N)
    word_ngrams_count = Counter(word_ngrams).most_common(50)
    authors.loc[author_id, "word_ngrams_count"] = json.dumps(word_ngrams_count)

authors[['username', 'words_count', 'byte_ngrams_count', 'word_ngrams_count']].head()

,username,words_count,byte_ngrams_count,word_ngrams_count
author_id,,,,
456226577798135808,Deleted User,"[[""the"", 21906], [""i"", 18328], [""a"", 17981], [...","[["" the"", 30394], [""ing "", 22350], [""the "", 21...","[[""egg egg egg"", 704], [""how do i"", 330], [""I ..."
651361636657135617,flattocatto,"[[""the"", 8834], [""to"", 6349], [""a"", 6103], [""I...","[["" the"", 11997], [""ing "", 9238], [""the "", 886...","[[""I don t"", 253], [""I can t"", 110], [""how to ..."
244966266845134858,buthed010203,"[[""the"", 9796], [""i"", 7382], [""it"", 7258], [""t...","[["" the"", 13312], [""the "", 9800], [""ing "", 795...","[[""a OMEGAROLL 650821998607597580"", 192], [""_ ..."
723940197456281640,archange3965,"[[""i"", 4798], [""the"", 3857], [""is"", 2636], [""a...","[["" the"", 5032], [""the "", 3860], [""ing "", 3346...","[[""i don t"", 205], [""i can t"", 119], [""don t k..."
559568621312147476,.deepfriedrice6828,"[[""the"", 6117], [""a"", 4007], [""I"", 3924], [""it...","[["" the"", 7811], [""the "", 6127], [""ing "", 5160...","[[""I don t"", 184], [""It s not"", 106], [""in pla..."


In [ ]:
#Garbage collection again
byte_ngrams = None
byte_ngrams_count = None
data_group = None
group = None
word_ngrams = None
word_ngrams_count = None
words = None
words_count = None
author_id = None
del byte_ngrams, byte_ngrams_count, data_group, group, word_ngrams, word_ngrams_count, words, words_count, author_id

In [ ]:
#Cleanup that has NaN, Inf
authors.dropna(inplace=True)

In [ ]:
#Save dataset
if not os.path.exists("dataset"):
    os.makedirs("dataset")
authors.to_csv("dataset/authors.csv", index=True)

# Resume from here

In [3]:
from os import listdir
from os.path import isfile, join
from collections import Counter

import os
import json
import pandas as pd
import math
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import string
import re

In [4]:
#Load dataset

authors = pd.read_csv("dataset/authors.csv")
authors.head()

,username,content,longest_message,sentences_sum,sentences_mean,sentences_median,words_sum,words_mean,words_median,punctuations_sum,...,byte_ngrams_median,word_ngrams_sum,word_ngrams_mean,word_ngrams_median,avg_sentence_length,avg_words_per_sentence,avg_punctuations_per_sentence,words_count,byte_ngrams_count,word_ngrams_count
0,Deleted User,190274,892,196409,1.032243,1.0,865550,4.548966,3.0,112389,...,14.0,537939,2.827181,1.0,1.032243,4.406875,0.572219,"[[""the"", 21906], [""i"", 18328], [""a"", 17981], [...","[["" the"", 30394], [""ing "", 22350], [""the "", 21...","[[""egg egg egg"", 704], [""how do i"", 330], [""I ..."
1,flattocatto,72738,975,73934,1.016443,1.0,334413,4.597501,4.0,20764,...,15.0,208410,2.865215,2.0,1.016443,4.523129,0.280845,"[[""the"", 8834], [""to"", 6349], [""a"", 6103], [""I...","[["" the"", 11997], [""ing "", 9238], [""the "", 886...","[[""I don t"", 253], [""I can t"", 110], [""how to ..."
2,buthed010203,53471,966,57685,1.078809,1.0,309528,5.788708,4.0,57263,...,22.0,214009,4.002338,2.0,1.078809,5.365832,0.992684,"[[""the"", 9796], [""i"", 7382], [""it"", 7258], [""t...","[["" the"", 13312], [""the "", 9800], [""ing "", 795...","[[""a OMEGAROLL 650821998607597580"", 192], [""_ ..."
3,archange3965,40709,354,41425,1.017588,1.0,145847,3.582672,2.0,15595,...,8.0,80056,1.966543,0.0,1.017588,3.520748,0.376463,"[[""i"", 4798], [""the"", 3857], [""is"", 2636], [""a...","[["" the"", 5032], [""the "", 3860], [""ing "", 3346...","[[""i don t"", 205], [""i can t"", 119], [""don t k..."
4,.deepfriedrice6828,38502,717,39281,1.020233,1.0,202842,5.268350,4.0,9079,...,17.0,134606,3.496078,2.0,1.020233,5.163871,0.231130,"[[""the"", 6117], [""a"", 4007], [""I"", 3924], [""it...","[["" the"", 7811], [""the "", 6127], [""ing "", 5160...","[[""I don t"", 184], [""It s not"", 106], [""in pla..."


In [5]:

def ValSim(x, y):
	"""
	Calculates the value similarity between two vectors x and y.
	
	Args:
	x (dict): A dictionary representing the first vector.
	y (dict): A dictionary representing the second vector.
	
	Returns:
	A float representing the value similarity between x and y.
	"""
	return (2*x*y)/(x**2+y**2)

def CosineSimIntersection(x, y):
	"""
	Calculates the cosine similarity between two vectors x and y using the intersection of their keys.
	
	Args:
	x (dict): A dictionary representing the first vector.
	y (dict): A dictionary representing the second vector.
	
	Returns:
	A float representing the cosine similarity between x and y.
	"""
	keys = set(x.keys()) & set(y.keys())

	num = sum([x[k] * y[k] for k in keys])
	sum1 = sum([x[k]**2 for k in keys])
	sum2 = sum([y[k]**2 for k in keys])
	denom = math.sqrt(sum1*sum2)

	if not denom:
		return 0.0
	return num/denom

def CosineSimUnion(x, y):
	"""
	Calculates the cosine similarity between two vectors x and y using the union of their keys.
	
	Args:
	x (dict): A dictionary representing the first vector.
	y (dict): A dictionary representing the second vector.
	
	Returns:
	A float representing the cosine similarity between x and y.
	"""
	keys = set(x.keys()) | set(y.keys())

	num = sum([x[k] * y[k] for k in keys])
	sum1 = sum([x[k]**2 for k in keys])
	sum2 = sum([y[k]**2 for k in keys])
	denom = math.sqrt(sum1*sum2)

	if not denom:
		return 0.0
	return num/denom

def JaccardSim(A, B):
	"""
	Calculates the Jaccard similarity between two sets A and B.
	
	Args:
	A (set): The first set.
	B (set): The second set.
	
	Returns:
	A float representing the Jaccard similarity between A and B.
	"""
	return len(A&B)/len(A|B)


In [29]:
#sort by the similiarity by avg_sentence_length	avg_words_per_sentence	avg_punctuations_per_sentence words_count	word_ngram_intersection byte_ngram_intersection byte_ngram_union
total_permutations = len(authors) * (len(authors) - 1) / 2
print("Total permutations: ", total_permutations)
similarity = pd.DataFrame(columns=["author1", "author2", "avg_sentence_length", "avg_words_per_sentence", "avg_punctuations_per_sentence", "words_count", "word_ngram_intersection", "byte_ngram_intersection", "byte_ngram_union", "average_similarity"])

Total permutations:  84246690.0


In [30]:
#set data type
similarity["author1"] = similarity["author1"].astype(str)
similarity["author2"] = similarity["author2"].astype(str)
similarity["words_count"] = similarity["words_count"].astype(float)
similarity["word_ngram_intersection"] = similarity["word_ngram_intersection"].astype(float)
similarity["byte_ngram_intersection"] = similarity["byte_ngram_intersection"].astype(float)
similarity["byte_ngram_union"] = similarity["byte_ngram_union"].astype(float)
similarity["average_similarity"] = similarity["average_similarity"].astype(float)

In [31]:
def save_similarity():
    global similarity
    if not os.path.exists("dataset"):
        os.makedirs("dataset")
    similarity.to_csv("dataset/similarity.csv", index=True)

In [34]:

last_time = time.time()
last_time_print = time.time()
last_pos = 0
opsPerSec = 0
timeLeft = "Unknown"

for source_author_i in authors.index:
    for target_author_i in authors.index:
        #avoid same author
        if source_author_i == target_author_i:continue

        #check if last time is more than 5 second
        if time.time() - last_time > 5:
            #calculate the operations per second
            opsPerSec = (len(similarity) - last_pos) / (time.time() - last_time)
            #update last time
            last_time = time.time()
            #update last position
            last_pos = len(similarity)
            #calculate the time left
            timeLeft = str(datetime.timedelta(seconds=math.ceil((total_permutations - len(similarity)) / opsPerSec)))
        
        index = str(hash(str(source_author_i)) + hash(str(target_author_i)))
        if index in similarity.index:continue

        source_author = authors.loc[source_author_i]
        target_author = authors.loc[target_author_i]
        avg_sentence_length = ValSim(source_author["avg_sentence_length"], target_author["avg_sentence_length"])
        avg_words_per_sentence = ValSim(source_author["avg_words_per_sentence"], target_author["avg_words_per_sentence"])
        avg_punctuations_per_sentence = ValSim(source_author["avg_punctuations_per_sentence"], target_author["avg_punctuations_per_sentence"])
        
        source_words_counter = Counter()
        target_words_counter = Counter()
        source_words_count_jason = json.loads(source_author["words_count"])
        target_words_count_jason = json.loads(target_author["words_count"])
        for word, count in source_words_count_jason:
            source_words_counter[word] = count
        for word, count in target_words_count_jason:
            target_words_counter[word] = count
        words_count = JaccardSim(source_words_counter, target_words_counter)

        source_word_ngrams_counter = Counter()
        target_word_ngrams_counter = Counter()
        source_word_ngrams_count_jason = json.loads(source_author["word_ngrams_count"])
        target_word_ngrams_count_jason = json.loads(target_author["word_ngrams_count"])
        for word, count in source_word_ngrams_count_jason:
            source_word_ngrams_counter[word] = count
        for word, count in target_word_ngrams_count_jason:
            target_word_ngrams_counter[word] = count
        word_ngram_intersection = CosineSimIntersection(source_word_ngrams_counter, target_word_ngrams_counter)

        source_byte_ngrams_counter = Counter()
        target_byte_ngrams_counter = Counter()
        source_byte_ngrams_count_jason = json.loads(source_author["byte_ngrams_count"])
        target_byte_ngrams_count_jason = json.loads(target_author["byte_ngrams_count"])
        for word, count in source_byte_ngrams_count_jason:
            source_byte_ngrams_counter[word] = count
        for word, count in target_byte_ngrams_count_jason:
            target_byte_ngrams_counter[word] = count
        byte_ngram_intersection = CosineSimIntersection(source_byte_ngrams_counter, target_byte_ngrams_counter)
        byte_ngram_union = CosineSimUnion(source_byte_ngrams_counter, target_byte_ngrams_counter)
        average_sim = (avg_sentence_length + avg_words_per_sentence + avg_punctuations_per_sentence + words_count + word_ngram_intersection + byte_ngram_intersection + byte_ngram_union) / 7
        
        source_author_username = source_author["username"]
        target_author_username = target_author["username"]

        similarity.loc[index] = [source_author_username, target_author_username, avg_sentence_length, avg_words_per_sentence, avg_punctuations_per_sentence, words_count, word_ngram_intersection, byte_ngram_intersection, byte_ngram_union, average_sim]

        #make the target author username length to 20, if less than 20, add spaces if more than 20, trim it
        target_author_username = target_author_username.ljust(20)[:20]
        


        if time.time() - last_time_print > 0.8:
            #print the progress
            #Source Author - Target Author - Similarity - Permutations - Percentage - Ops/sec - Time Left
            print(f"{source_author_username} - {target_author_username} - {max(0, min(100, average_sim * 100)):.2f}% - {len(similarity)}/{total_permutations} ({len(similarity)/total_permutations*100:.2f}%) - {opsPerSec:.2f} ops/sec - {timeLeft} left"
            , end="                         \r")
            #update last time print
            last_time_print = time.time()
    save_similarity()
save_similarity()
    


KeyboardInterrupt: 

In [18]:
save_similarity()

# Resume from here

In [ ]:
import pandas as pd

In [ ]:
#Load the similarity dataset
similarity = pd.read_csv("dataset/similarity.csv")

In [36]:
similarity.sort_values(by=["average_similarity"], ascending=False, inplace=True)
similarity.head()

,author1,author2,avg_sentence_length,avg_words_per_sentence,avg_punctuations_per_sentence,words_count,word_ngram_intersection,byte_ngram_intersection,byte_ngram_union,average_similarity
7444755801254930229,Deleted User,jazzmuse,0.999973,0.983239,0.999987,0.785714,0.982193,0.990000,0.929032,0.952877
2005696787223104106,Deleted User,chips,0.999978,0.998823,0.998710,0.785714,0.879844,0.990419,0.934075,0.941081
-4663263947830671770,Deleted User,thecoolestboi9499,0.999944,0.952176,0.993627,0.785714,0.929156,0.994125,0.928989,0.940533
15745501797732654461,flattocatto,.deepfriedrice6828,0.999993,0.991288,0.981317,0.724138,0.951966,0.992697,0.936899,0.939757
-8949230577872434900,Deleted User,drago4370,0.999991,0.972937,0.972819,0.785714,0.930036,0.993631,0.917526,0.938951
